In [1]:
import os
import time 
import random
import pygame
import neat


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#define the size of the window
pygame.font.init()
WIN_WIDTH = 500
WIN_HEIGHT = 800
STAT_FONT = pygame.font.SysFont("comicsans", 40)
#load the images of birds
BIRD_IMGS = [pygame.transform.scale2x(pygame.image.load(os.path.join("./imgs", "bird1.png"))),\
        pygame.transform.scale2x(pygame.image.load(os.path.join("./imgs", "bird2.png"))),\
        pygame.transform.scale2x(pygame.image.load(os.path.join("./imgs", "bird3.png")))]
#setup the font for displaying the score


In [3]:
#load the pipe, base and background images
PIPE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("./imgs", "pipe.png")))
BASE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("./imgs", "base.png")))
BGRD_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("./imgs", "bg.png")))

In [10]:
class Bird:
    IMGS = BIRD_IMGS
    MAX_ROTATION = 25
    ROT_VELOCITY = 20
    ANIMATION = 5
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.tilt = 0
        self.height = y
        self.tick_count = 0
        self.velocity = 0
        self.img_count = 0
        self.img = self.IMGS[0]
        
    def jump(self):
        self.velocity = -10.5
        self.tick_count = 0
        self.height = self.y
        
    def move(self):
        self.tick_count += 1      
        d = self.velocity*self.tick_count + 1.5*self.tick_count**2
        if d >= 16:
            d = 16
        elif d < 0:
            d -= 2
        self.y += d    
        if d < 0 or self.y < self.height + 50:
            if self.tilt < self.MAX_ROTATION:
                self.tilt = self.MAX_ROTATION
        else:
            if self.tilt > -90:
                self.tilt -= self.ROT_VELOCITY
                
    def draw(self, win):
        self.img_count += 1
        
        if self.img_count < self.ANIMATION:
            self.img = self.IMGS[0]
        elif self.img_count < self.ANIMATION*2:
            self.img = self.IMGS[1]
        elif self.img_count < self.ANIMATION*3:
            self.img = self.IMGS[2]
        elif self.img_count < self.ANIMATION*4:
            self.img = self.IMGS[1]
        elif self.img_count < self.ANIMATION*4 + 1:
            self.img = self.IMGS[0]
            self.img_count = 0
        
        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.img_count = self.ANIMATION * 2
        rotated_image = pygame.transform.rotate(self.img, self.tilt)
        new_rect = rotated_image.get_rect(center=self.img.get_rect(topleft=(self.x, self.y)).center)
        
        win.blit(rotated_image, new_rect.topleft)
        
    def get_mask(self):
        return pygame.mask.from_surface(self.img)
    
class Pipe:
    GAP = 200
    VEL = 5
    def __init__(self, x):
        self.x = x
        self.height = 0
        self.top = 0
        self.bottom = 0
        self.PIPE_TOP = pygame.transform.flip(PIPE_IMG, False, True)
        self.PIPE_BOTTOM = PIPE_IMG
        self.passed = False    
        self.set_height()
        
    def set_height(self):
        self.height = random.randrange(50, 450)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height + self.GAP
        
    def move(self):
        self.x -= self.VEL
        
    def draw(self, win):
        win.blit(self.PIPE_TOP, (self.x, self.top))
        win.blit(self.PIPE_BOTTOM, (self.x, self.bottom))
    
    def collide(self, bird):
        bird_mask = bird.get_mask()
        top_mask = pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)
        
        top_offset = (self.x - bird.x, self.top - round(bird.y))
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))
        
        b_point = bird_mask.overlap(bottom_mask, bottom_offset)
        t_point = bird_mask.overlap(top_mask, top_offset)
        if b_point or t_point:
            return True
        return False
        
class Base: 
    VEL = 5
    WIDTH = BASE_IMG.get_width()
    IMG = BASE_IMG
    def __init__(self, y):
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH
    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL
        
        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH
            
        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH
    def draw(self, win):
        win.blit(self.IMG, (self.x1, self.y))
        win.blit(self.IMG, (self.x2, self.y))
        
def draw_window(win, birds, pipes, base, score):
    win.blit(BGRD_IMG, (0,0))
    for pipe in pipes:
        pipe.draw(win)
    text = STAT_FONT.render("Score: " + str(score), 1, (255, 255, 255))
    win.blit(text, (WIN_WIDTH - 10 - text.get_width(), 10))
    base.draw(win)
    for bird in birds:
        bird.draw(win)
    pygame.display.update()
    
def main(genomes, config):
    #bird = Bird(230, 350)
    ge = []
    nets = []
    birds = []
    
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        birds.append(Bird(230, 350))
        g.fitness = 0
        ge.append(g)
        
    
    base = Base(730)
    pipes = [Pipe(600)]
    win = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
    pygame.display.set_caption('Flappy Bird')
    clock = pygame.time.Clock()
    run = True
    score = 0
    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.display.quit()

        pipe_ind = 0
        
        if len(birds) > 0:
            if len(pipes) > 1 and birds[0].x > pipes[0].x + pipes[0].PIPE_TOP.get_width():
                pipe_ind = 1
        else:
            run = False
            break
                
        for x, bird in enumerate(birds):
            bird.move()
            ge[x].fitness += 0.1
            output = nets[x].activate((bird.y, abs(bird.y - pipes[pipe_ind].height), abs(bird.y - pipes[pipe_ind].bottom)))
            if output[0] > 0.5:
                bird.jump()
        
        rem = []
        add_pipe = False
        for pipe in pipes:
            for x, bird in enumerate(birds):
                if pipe.collide(bird):
                    ge[x].fitness -= 0.5
                    
                    #remove a bird if it hits the pipe (and die) 
                    birds.pop(x)
                    nets.pop(x)
                    ge.pop(x)
                    
                if not pipe.passed and pipe.x < bird.x: 
                    pipe.passed = True
                    add_pipe = True
            if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                rem.append(pipe)
            pipe.move()
        if add_pipe:
            score += 1
            for g in ge:
                g.fitness += 0.5
            pipes.append(Pipe(600))
        for x, bird in enumerate(birds):
            if bird.y + bird.img.get_height() >= 730 or bird.y < 0: 
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
        for r in rem:
            rem.remove(r)
        base.move()
        draw_window(win, birds, pipes, base, score)
        #bird.move()
    
    #pygame.display.flip()
    #quit()
    
def run(config_path):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                               neat.DefaultSpeciesSet, neat.DefaultStagnation,
                               config_path) 
    
    p = neat.Population(config)
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    
    winner = p.run(main, 20)

if __name__ == "__main__":
    #local_dir = os.path.dirname(__file__)
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward.txt")
    run(config_path)
    
     


 ****** Running generation 0 ****** 

Population's average fitness: 4.29000 stdev: 2.89671
Best fitness: 15.20000 - size: (1, 3) - species 1 - id 8
Average adjusted fitness: 0.154
Mean genetic distance 1.071, standard deviation 0.446
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    20     15.2    0.154     0
Total extinctions: 0
Generation time: 5.197 sec

 ****** Running generation 1 ****** 

Population's average fitness: 4.88500 stdev: 4.24444
Best fitness: 22.00000 - size: (1, 3) - species 1 - id 8
Average adjusted fitness: 0.131
Mean genetic distance 1.259, standard deviation 0.546
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    20     22.0    0.131     0
Total extinctions: 0
Generation time: 7.355 sec (6.276 average)

 ****** Running generation 2 ****** 

Population's average fitness: 5.49500 stdev: 5.85

error: display Surface quit